# Installation: 安裝套件


In [ ]:
!pip install colabcam

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 41.3 MB/s eta 0:00:00
  Created wheel for colabcam: filename=colabcam-1.0.3-py3-none-any.whl size=5487 sha256=c07597599af1a024561b7e379fe7df0b7acd5f1925596c3e747112798929cc06
  Stored in directory: /root/.cache/pip/wheels/5a/ec/08/a87ee87cbf9c5d825e3433eb730e708d708e02158cd56f9606
Successfully built colabcam


In [ ]:
!pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 25.6 MB/s eta 0:00:00


# Build function: 公式讀取與參數指定

In [27]:
from anpr_easy import PyImageSearchANPR
import easyocr
import numpy as np
import cv2

reader = easyocr.Reader(['en'])
def main(image,contour_box_ratio = 0.6,keep = 5,minAR = 1.5,maxAR = 4,license_num = 6,psm = 7,verbose = None):
    aa = PyImageSearchANPR(reader,minAR=minAR, maxAR=maxAR,license_num=license_num,verbose = verbose,keep = keep, contour_box_ratio = contour_box_ratio)
    result= aa.find_and_ocr(image, psm=psm)
    if type(result)==tuple:
        (lpText, lpCnt) = result
        return cv2.boundingRect(lpCnt),lpText.strip()

def realtime_process(img):
    bgra = np.zeros([img.shape[0],img.shape[1],4], dtype=np.uint8)
    result = main(img)
    if result!=None:
        (x,y,w,h),text = result
        bgra = cv2.rectangle(bgra,(x,y),(x+w,y+h),(255,0,0),2)
        bgra = cv2.putText(bgra,text,(x,y - 15),cv2.FONT_HERSHEY_SIMPLEX,0.75,(255,0,0),2)
        bgra[:,:,3] = (bgra.max(axis = 2) > 0 ).astype(int) * 255
        return bgra,text
    else:
        return bgra,''

# Run: 運行

In [ ]:
import colab_cam as cc

cc.video_stream()
label_html = 'Capturing...'
overlap_img = ''

while True:

    js_reply = cc.video_frame(label_html, overlap_img)

    if not js_reply:
        break

    img = colabcam.js_to_image(js_reply["img"])

    overlap_array,t = realtime_process(img)
    overlap_bytes = cc.overlap_to_bytes(overlap_array)
    overlap_img = overlap_bytes

<IPython.core.display.Javascript object>